<a href="https://colab.research.google.com/github/JoungMinJu/lab_study/blob/main/%EC%9B%8C%EB%93%9C_%EC%9E%84%EB%B2%A0%EB%94%A9%EC%9D%98_%ED%8F%89%EA%B7%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

앞서 특정 문장 내의 단어들의 임베딩 벡터 들의 평균이 그 문장의 벡터가 될 수 있음을 설명했다.

여기에서는 임베딩이 잘 된 상황에서는 단어 벡터들의 평균만으로 텍스트 분류를 수행할 수 있음을 보이고, 워드 임베딩의 중요성을 이해해 볼 것이다.


영화사이트 IMDB 영화 리뷰 데이터는 리뷰가 긍정 1 부정 0으로 레이블링한 데이터로 25,000개의 훈련데이터와 테스트데이터 25,000개로 구성되었음


In [2]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import imdb

In [3]:
vocab_size = 20000

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수 :',len(x_train))
print('테스트용 리뷰 개수 :',len(x_test))


#num_words 사용하면 이 데이터에서 등장 빈도 순위로 몇 번째에 해당하는 단어까지 사용할 것인지를 지정할 수 있음
#


17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [4]:
#이 데이터는 이미 정수 인코딩까지의 전처리가 진행된 상태

print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
print(y_train[0])

1


In [8]:
print('훈련용 리뷰의 평균 길이: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('테스트용 리뷰의 평균 길이: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

훈련용 리뷰의 평균 길이: 238
테스트용 리뷰의 평균 길이: 230


In [9]:
#데이터 패딩
max_len = 400

x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)
print('x_train의 크기(shape) :', x_train.shape)
print('x_test의 크기(shape) :', x_test.shape)

x_train의 크기(shape) : (25000, 400)
x_test의 크기(shape) : (25000, 400)


In [10]:
#모델 설계하기

#임베딩 벡터를 평균으로 사용하는 모델을 설계
#GlobalAveragePooling1D()는 입력으로 들어오는 모든 벡터들의 평균을 구하는 역할을 한다.
#따라서 Embedding()다음에  Global...()을 추가하면 해당 문장의 모든 단어 벡터들의 평균 벡터를 구하게 됨

#이진분류 수행해야하므로 그 후에는 시그모이드 함수를 활성화 함수로 사용하는 뉴런 한개를 배치한다.

model=Sequential()
model.add(Embedding(vocab_size,50, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))


In [13]:
es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc=ModelCheckpoint('embedding_average_model.h5',monitor='val_acc', mode='max',verbose=1,save_best_only=True)

In [14]:
#훈련데이터의 20%를 검증 데이터로 사용하고 총 10회 학습
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(x_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc], validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 11s 17ms/step - loss: 0.6351 - acc: 0.7150 - val_loss: 0.5377 - val_acc: 0.8084

Epoch 00001: val_acc improved from -inf to 0.80840, saving model to embedding_average_model.h5
Epoch 2/10
625/625 [==============================] - 11s 18ms/step - loss: 0.4492 - acc: 0.8493 - val_loss: 0.3971 - val_acc: 0.8648

Epoch 00002: val_acc improved from 0.80840 to 0.86480, saving model to embedding_average_model.h5
Epoch 3/10
625/625 [==============================] - 11s 18ms/step - loss: 0.3357 - acc: 0.8862 - val_loss: 0.3354 - val_acc: 0.8762

Epoch 00003: val_acc improved from 0.86480 to 0.87620, saving model to embedding_average_model.h5
Epoch 4/10
625/625 [==============================] - 11s 18ms/step - loss: 0.2764 - acc: 0.9050 - val_loss: 0.3052 - val_acc: 0.8848

Epoch 00004: val_acc improved from 0.87620 to 0.88480, saving model to embedding_average_model.h5
Epoch 5/10
625/625 [==============================] - 11s 17ms/step - l

In [15]:
#테스트 데이터에 대해서 평가
loaded_model = load_model('embedding_average_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

782/782 [==============================] - 2s 3ms/step - loss: 0.2865 - acc: 0.8874

 테스트 정확도: 0.8874


In [16]:
#별다른 신경망을 추가하지 않고도 단어 벡터의 평균마으로도 88.76%라는 높은 정확도를 얻어냄